In [119]:
import config

In [120]:
import spotipy
import json
import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from pyjarowinkler import distance
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.c_id,
                                                           client_secret= config.c_se))

### Getting playlists

In [121]:
# locating playlist
playlist_country = sp.user_playlist_tracks("spotify", "3RPaWvjgg1SAi8oIh09Irp",market="GB")
playlist_rock = sp.user_playlist_tracks("spotify", "7KMtxb9kh3TtVvJTdLtJ2K",market="GB")
playlist_house = sp.user_playlist_tracks("spotify", "4VXFwpNvQysB8yS6E2t2ls",market="GB")
playlist_classicals = sp.user_playlist_tracks("spotify", "1h0CEZCm6IbFTbxThn6Xcs",market="GB")
playlist_dance = sp.user_playlist_tracks("spotify", "794tEP1QrKMcaQcFmEyFst",market="GB")
playlist_hiphop = sp.user_playlist_tracks("spotify", "0agbFCQZvK4q9WR0d4Iqcj",market="GB")
playlist_pop = sp.user_playlist_tracks("spotify", "2YRe7HRKNRvXdJBp9nXFza",market="GB")
playlist_jazz = sp.user_playlist_tracks("spotify", "7g2KyiE9RI5F1fV60OIUu4",market="GB")
playlist_kpop = sp.user_playlist_tracks("spotify", "37i9dQZF1DX14fiWYoe7Oh",market="GB")
playlist_rnb = sp.user_playlist_tracks("spotify", "37i9dQZF1DX04mASjTsvf0",market="GB")
playlist_metal = sp.user_playlist_tracks("spotify", "37i9dQZF1DX9qNs32fujYe",market="GB")
playlist_techno = sp.user_playlist_tracks("spotify", "37i9dQZF1DX6J5NfMJS675",market="GB")

### Foundation for tracks

In [122]:
track=[]

In [123]:
# extracting tracks from playlist
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="GB")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

### Foundation for artists

In [124]:
artist=[]

In [125]:
# extracting artists from track
def get_artists_from_track(track):
    return [artist["name"] for artist in track["artists"]]

In [126]:
# extracting artists from playlist
def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("spotify", playlist_id)
    return list(get_artists_from_track(track["track"]) for track in tracks_from_playlist)

### Foundation for track ids

In [127]:
track_id=[]

### Combining data

In [128]:
# creating a dataframe
track_df = pd.DataFrame({'title':track,
                     'artist':artist,
                     'id':track_id})

In [129]:
# function to call tracks, artists and track ids from a playlist
def append_info(playlist_id):
    global track_df
    new_track_list = get_playlist_tracks("spotify", playlist_id)
    new_artist_list = get_artists_from_playlist(playlist_id)
    new_track=[]
    new_artist=[]
    new_track_id=[]
    for i in range(0, len(new_track_list)):
        new_track.append(new_track_list[i]['track']['name'])
    for i in new_artist_list:
        new_artist.append(", ".join(i))
    for i in range(0, len(new_track_list)):
        new_track_id.append(new_track_list[i]['track']['id'])
    new_df = pd.DataFrame({'title':new_track,
                          'artist':new_artist,
                          'id':new_track_id
                          })
    track_df = track_df.append(new_df)    

In [130]:
append_info("3RPaWvjgg1SAi8oIh09Irp")

In [131]:
append_info("7KMtxb9kh3TtVvJTdLtJ2K")

In [132]:
append_info("4VXFwpNvQysB8yS6E2t2ls")

In [133]:
append_info("1h0CEZCm6IbFTbxThn6Xcs")

In [134]:
append_info("794tEP1QrKMcaQcFmEyFst")

In [135]:
append_info("0agbFCQZvK4q9WR0d4Iqcj")

In [136]:
append_info("2YRe7HRKNRvXdJBp9nXFza")

In [137]:
append_info("7g2KyiE9RI5F1fV60OIUu4")

In [138]:
append_info("37i9dQZF1DX14fiWYoe7Oh")

In [139]:
append_info("37i9dQZF1DX04mASjTsvf0")

In [140]:
append_info("37i9dQZF1DX9qNs32fujYe")

In [141]:
append_info("37i9dQZF1DX6J5NfMJS675")

In [142]:
# checking duplicates
track_df[track_df.id.duplicated(keep=False)].sort_values(by =['id'])

,title,artist,id
81,Mr. Brightside,The Killers,003vvx7Niy0yvhvHt4a68B
21,Mr. Brightside,The Killers,003vvx7Niy0yvhvHt4a68B
2693,Grits,RZA,00762tXbSj2w2bQMVydJZn
275,Grits,RZA,00762tXbSj2w2bQMVydJZn
1660,Ain't No Future In Yo' Frontin',"MC Breed, DFC",00ReeHCY0FQUyuAUyPJdnk
...,...,...,...
185,If I Ruled The World (Imagine That),Nas Feat. Lauryn Hill,None
338,Makin' Moves,Kirk,None
339,Walk Thru Hell (Street Version),Madman Shawn,None
1601,Tic Toc,Lords Of The Underground,None


In [143]:
# dropping duplicates
track_df.drop_duplicates(subset=['id'],keep='first',inplace=True)

In [144]:
# resetting index
track_df.reset_index(inplace=True)

In [145]:
# dropping index column
track_df.drop(columns=['index'],inplace=True)

In [146]:
track_df

,title,artist,id
0,Crazy - Single Version,"Patsy Cline, The Jordanaires",3zpj9dvJABiyMrmLCPw6i8
1,Folsom Prison Blues,"Johnny Cash, The Tennessee Two",121sjkub4p4zcFNozkKnQD
2,I Will Always Love You,Dolly Parton,6nRvgo4ySezQKD1D7NIqAu
3,Springsteen,Eric Church,05ZACkzW8YbRq3eFgIfSNB
4,He Stopped Loving Her Today,George Jones,6qMgx8ys4MNNxypaIvbG0i
...,...,...,...
8965,Afterhour,Carlo Ruetz,2Sh9tvtvR7rmnBpeQ7KbPu
8966,Latex,Monika Kruse,7Lp0NobaOnTfqcHPjVXzA6
8967,Whisper,Anna Reusch,0j8iHPirDSkD5nUQw9XzRF
8968,Mother Earth - Original Mix,Fatima Hajji,4U29PAbl99xuzW3JWSX0u0


In [147]:
features_df = pd.DataFrame(columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                   'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                   'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
                                   'time_signature'])
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature


In [150]:
for i in range(100,1000):
    features_id = 'spotify:track:'+track_df.id[i]
    my_dict = sp.audio_features(features_id)[0]
    my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
    features_df_new = pd.DataFrame(my_dict_new)
    features_df = features_df.append(features_df_new)
    

In [149]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.471,0.021,10,-24.385,1,0.0366,0.9310,0.000001,0.1030,0.356,107.796,audio_features,3zpj9dvJABiyMrmLCPw6i8,spotify:track:3zpj9dvJABiyMrmLCPw6i8,https://api.spotify.com/v1/tracks/3zpj9dvJABiy...,https://api.spotify.com/v1/audio-analysis/3zpj...,164573,3
0,0.754,0.540,5,-8.326,1,0.0266,0.6870,0.000075,0.0868,0.790,101.738,audio_features,121sjkub4p4zcFNozkKnQD,spotify:track:121sjkub4p4zcFNozkKnQD,https://api.spotify.com/v1/tracks/121sjkub4p4z...,https://api.spotify.com/v1/audio-analysis/121s...,168554,4
0,0.446,0.142,9,-13.498,1,0.0335,0.9600,0.001140,0.1010,0.156,135.724,audio_features,6nRvgo4ySezQKD1D7NIqAu,spotify:track:6nRvgo4ySezQKD1D7NIqAu,https://api.spotify.com/v1/tracks/6nRvgo4ySezQ...,https://api.spotify.com/v1/audio-analysis/6nRv...,176093,4
0,0.692,0.809,2,-5.935,1,0.0255,0.1000,0.000098,0.1850,0.927,104.023,audio_features,05ZACkzW8YbRq3eFgIfSNB,spotify:track:05ZACkzW8YbRq3eFgIfSNB,https://api.spotify.com/v1/tracks/05ZACkzW8YbR...,https://api.spotify.com/v1/audio-analysis/05ZA...,263387,4
0,0.430,0.173,8,-18.597,1,0.0321,0.5940,0.000111,0.2240,0.365,72.103,audio_features,6qMgx8ys4MNNxypaIvbG0i,spotify:track:6qMgx8ys4MNNxypaIvbG0i,https://api.spotify.com/v1/tracks/6qMgx8ys4MNN...,https://api.spotify.com/v1/audio-analysis/6qMg...,195333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.507,0.850,5,-5.679,1,0.0472,0.3780,0.000000,0.0567,0.847,171.800,audio_features,1zG6cEmeo4rv7Old8XD7vC,spotify:track:1zG6cEmeo4rv7Old8XD7vC,https://api.spotify.com/v1/tracks/1zG6cEmeo4rv...,https://api.spotify.com/v1/audio-analysis/1zG6...,183733,4
0,0.679,0.557,2,-10.482,1,0.0378,0.8260,0.000006,0.9590,0.769,119.060,audio_features,6wAD64HhuMcXGA4Z64BLqq,spotify:track:6wAD64HhuMcXGA4Z64BLqq,https://api.spotify.com/v1/tracks/6wAD64HhuMcX...,https://api.spotify.com/v1/audio-analysis/6wAD...,137600,4
0,0.679,0.553,3,-14.687,1,0.0316,0.0611,0.000007,0.1390,0.946,109.071,audio_features,064SVQsmWl5EF0zahmzkQk,spotify:track:064SVQsmWl5EF0zahmzkQk,https://api.spotify.com/v1/tracks/064SVQsmWl5E...,https://api.spotify.com/v1/audio-analysis/064S...,198760,4
0,0.693,0.324,11,-15.085,1,0.2050,0.7960,0.000000,0.2460,0.382,144.140,audio_features,3P6OwCX7Ofiaaqtvujb6i5,spotify:track:3P6OwCX7Ofiaaqtvujb6i5,https://api.spotify.com/v1/tracks/3P6OwCX7Ofia...,https://api.spotify.com/v1/audio-analysis/3P6O...,158307,4


In [83]:
test_song = 'spotify:track:'+track_df.iloc[0][2]

In [104]:
my_dict = sp.audio_features(test_song)[0]
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
features_df = pd.DataFrame(my_dict_new)
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.471,0.021,10,-24.385,1,0.0366,0.931,0.000001,0.103,0.356,107.796,audio_features,3zpj9dvJABiyMrmLCPw6i8,spotify:track:3zpj9dvJABiyMrmLCPw6i8,https://api.spotify.com/v1/tracks/3zpj9dvJABiy...,https://api.spotify.com/v1/audio-analysis/3zpj...,164573,3


In [82]:
test_song2 = 'spotify:track:'+track_df.iloc[1][2]

In [85]:
my_dict2 = sp.audio_features(test_song2)[0]
my_dict_new2 = { key: [my_dict2[key]] for key in list(my_dict2.keys()) }
#features_df2 = pd.DataFrame(my_dict_new)
#features_df2

In [79]:
features_df.append(features_df2)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.471,0.021,10,-24.385,1,0.0366,0.931,0.000001,0.1030,0.356,107.796,audio_features,3zpj9dvJABiyMrmLCPw6i8,spotify:track:3zpj9dvJABiyMrmLCPw6i8,https://api.spotify.com/v1/tracks/3zpj9dvJABiy...,https://api.spotify.com/v1/audio-analysis/3zpj...,164573,3
0,0.754,0.540,5,-8.326,1,0.0266,0.687,0.000075,0.0868,0.790,101.738,audio_features,121sjkub4p4zcFNozkKnQD,spotify:track:121sjkub4p4zcFNozkKnQD,https://api.spotify.com/v1/tracks/121sjkub4p4z...,https://api.spotify.com/v1/audio-analysis/121s...,168554,4


In [107]:
features_id5 = 'spotify:track:'+track_df.id[1]

In [108]:
my_dict5 = sp.audio_features(features_id5)[0]

In [109]:
my_dict_new5 = { key: [my_dict5[key]] for key in list(my_dict5.keys()) }

In [117]:
features_df5 = pd.DataFrame(my_dict_new5)
features_df5.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [111]:
features_df.append(features_df5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.471,0.021,10,-24.385,1,0.0366,0.931,0.000001,0.1030,0.356,107.796,audio_features,3zpj9dvJABiyMrmLCPw6i8,spotify:track:3zpj9dvJABiyMrmLCPw6i8,https://api.spotify.com/v1/tracks/3zpj9dvJABiy...,https://api.spotify.com/v1/audio-analysis/3zpj...,164573,3
0,0.754,0.540,5,-8.326,1,0.0266,0.687,0.000075,0.0868,0.790,101.738,audio_features,121sjkub4p4zcFNozkKnQD,spotify:track:121sjkub4p4zcFNozkKnQD,https://api.spotify.com/v1/tracks/121sjkub4p4z...,https://api.spotify.com/v1/audio-analysis/121s...,168554,4


In [114]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.471,0.021,10,-24.385,1,0.0366,0.931,0.000001,0.1030,0.356,107.796,audio_features,3zpj9dvJABiyMrmLCPw6i8,spotify:track:3zpj9dvJABiyMrmLCPw6i8,https://api.spotify.com/v1/tracks/3zpj9dvJABiy...,https://api.spotify.com/v1/audio-analysis/3zpj...,164573,3
0,0.754,0.540,5,-8.326,1,0.0266,0.687,0.000075,0.0868,0.790,101.738,audio_features,121sjkub4p4zcFNozkKnQD,spotify:track:121sjkub4p4zcFNozkKnQD,https://api.spotify.com/v1/tracks/121sjkub4p4z...,https://api.spotify.com/v1/audio-analysis/121s...,168554,4
0,0.446,0.142,9,-13.498,1,0.0335,0.960,0.001140,0.1010,0.156,135.724,audio_features,6nRvgo4ySezQKD1D7NIqAu,spotify:track:6nRvgo4ySezQKD1D7NIqAu,https://api.spotify.com/v1/tracks/6nRvgo4ySezQ...,https://api.spotify.com/v1/audio-analysis/6nRv...,176093,4
0,0.692,0.809,2,-5.935,1,0.0255,0.100,0.000098,0.1850,0.927,104.023,audio_features,05ZACkzW8YbRq3eFgIfSNB,spotify:track:05ZACkzW8YbRq3eFgIfSNB,https://api.spotify.com/v1/tracks/05ZACkzW8YbR...,https://api.spotify.com/v1/audio-analysis/05ZA...,263387,4
0,0.430,0.173,8,-18.597,1,0.0321,0.594,0.000111,0.2240,0.365,72.103,audio_features,6qMgx8ys4MNNxypaIvbG0i,spotify:track:6qMgx8ys4MNNxypaIvbG0i,https://api.spotify.com/v1/tracks/6qMgx8ys4MNN...,https://api.spotify.com/v1/audio-analysis/6qMg...,195333,4


In [115]:
features_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [88]:
test_list.append(my_dict_new)

In [90]:
test_list.append(my_dict_new2)

In [92]:
pd.DataFrame(test_list)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,[0.471],[0.021],[10],[-24.385],[1],[0.0366],[0.931],[1.19e-06],[0.103],[0.356],[107.796],[audio_features],[3zpj9dvJABiyMrmLCPw6i8],[spotify:track:3zpj9dvJABiyMrmLCPw6i8],[https://api.spotify.com/v1/tracks/3zpj9dvJABi...,[https://api.spotify.com/v1/audio-analysis/3zp...,[164573],[3]
1,[0.754],[0.54],[5],[-8.326],[1],[0.0266],[0.687],[7.51e-05],[0.0868],[0.79],[101.738],[audio_features],[121sjkub4p4zcFNozkKnQD],[spotify:track:121sjkub4p4zcFNozkKnQD],[https://api.spotify.com/v1/tracks/121sjkub4p4...,[https://api.spotify.com/v1/audio-analysis/121...,[168554],[4]


In [41]:
test_song = 'spotify:track:'+track_df.iloc[0][2]
test_song

'spotify:track:3zpj9dvJABiyMrmLCPw6i8'

In [184]:
# saving dataframe to a csv
from pathlib import Path  
filepath = Path('track_df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
track_df.to_csv(filepath, index=False)